In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('../..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

In [ ]:
from neuron.neurons import Neuron, User
from neuron.neurons import Neuron
from neuron.capabilities import DataFrameRetrieverCapability, RerankCapability
from neuron.components import SequentialComponent, CycleComponent, Pipeline

# Start logging
from neuron.runtime_logging import start
logging_session_id = start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

In [ ]:
import pandas as pd
df = pd.read_json(r"./data/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=1000000)
# read the data 
for business in df:
    business = business.head(100)
    break

business.drop(['business_id', 'is_open', 'hours', 'longitude', 'latitude', 'postal_code', 'state', 'city', 'attributes'], axis=1, inplace=True)
business.rename(columns={'name': 'place_name'}, inplace=True)

business['categories'] = business['categories'].str.split(', ')
business = business.explode('categories', ignore_index=True)

In [ ]:
business.head()

### Conversational neuron

In [ ]:
llm_config={
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-groq-70b-8192-tool-use-preview",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    name="User",
    llm_config=llm_config,
)

extract_info_neuron = Neuron(
    name="extract_info_neuron",
    llm_config=llm_config,
    system_message="""
    You are an AI assistant tasked with extracting specific features from a user request. Your role is to identify and include only the features that are explicitly mentioned or can be directly inferred from the provided text. 

    ### Features to Extract:
    1. 'stars' (ratings) as a numeric value.
    2. 'review_count' as a numeric value.
    3. 'categories' describing the business type. If not explicitly mentioned, infer from context or exclude it if unclear.

    ### Output Format:
    Include only the features that are explicitly mentioned or inferable, formatted as:
    - stars is {stars_value}
    - review_count is {review_count_value}
    - categories is {categories_value}

    ### Rules:
    1. Include only the features described or inferable in the input. If a feature is not mentioned or cannot be inferred, do not include it in the output.
    2. Provide only the extracted details. Do not leave placeholders or add features with empty values.
    3. Do not include comments, explanations, or extra text in the output.

    ### Examples:
    1. Input: "I want to eat pizza at a restaurant with at least 3 stars and more than 30 reviews."
    Output: 
    stars is 3
    review_count is 30
    categories is Restaurant

    2. Input: "I need a clinic with at least 4 stars."
    Output: 
    stars is 4
    categories is Health & Medical

    3. Input: "Looking for a nice park with good views."
    Output: 
    categories is Park

    4. Input: "I need something fun to do."
    Output: 
    (No output)
    """,
    shared_memory_write_keys = ["extract_info"],
)

retriever_neuron = Neuron(
    name="retriever_neuron",
    llm_config=llm_config, 
    system_message="""
    Format the retrieved items into a Python list of strings. Each string should include all item details in the format:

    [
        'Item 1: address is 123 Main St, categories is Shopping, place_name is Example Store, review_count is 10, stars is 4.0',
        'Item 2: address is 456 Elm St, categories is Food, place_name is Example Cafe, review_count is 15, stars is 4.5'
    ]

    Output only the Python list of strings with all item details.
    Remove any additional text, comments, or information. I mean, leave only the list of strings.
    """
)
data_frame_retriever_capability = DataFrameRetrieverCapability(
    dataset=business, 
    columns = ["stars" , "review_count", "categories"],
    cache=True,
    top_n=10,
    config= {
        "model": "gpt2-medium",
        "model_dir": r"./models/erasmo_yelp_gpt2-medium_60_True"
    },
)
data_frame_retriever_capability.add_to_neuron(retriever_neuron)

rerank_neuron = Neuron(
    name="rerank_neuron",
    llm_config=llm_config,
    system_message="""
        You will be provided a list of item which were reranked. 
        Your goal is to create a short introduction about the list to the user.
        Also, include all items in the list in the output.
    """,
)
rerank_capability = RerankCapability(
    rerank_model_name = "rerank-english-v2.0", #"rerank-v3.5",
    api_key=os.getenv("COHERE_API_KEY"),
    top_n=10,
    shared_memory_read_keys=["extract_info"],
)
rerank_capability.add_to_neuron(rerank_neuron)

assessor_neuron = Neuron(
    name="assessor_neuron",
    llm_config=llm_config, 
    system_message="""
You are an Assessor assistant tasked with evaluating whether each retrieved item matches the user's specified criteria for `stars`, `review count`, and `category`.

### Instructions:

1. Understand the User's Criteria:
   - Identify the minimum requirements for `stars`, `review count`, and `category`.

2. Evaluate Each Item:
   - For each retrieved item, check:
     - Does the `stars` value meet or exceed the user's requirement?
     - Does the `review count` value meet or exceed the user's requirement?
     - Does the `category` match the user's specified category?
   - Respond for each item with:
     - The item itself, followed by `"Matches the user's criteria."` if it meets all requirements.
     - The item itself, followed by `"Does not match the user's criteria."` if it fails any requirement.

3. Output:
   - List each retrieved item alongside its evaluation in the format:
     ```plaintext
     Item: <item_details> - Matches the user's criteria.
     Item: <item_details> - Does not match the user's criteria.
     ```
   - Ensure all items are included in the output.
    """,
    shared_memory_read_keys = ["extract_info"],
    shared_memory_transition_message = ["Below is the user's request:"]
)

recommender = Neuron(
    name="recommender",
    llm_config=llm_config,
    system_message="""
    As a Recommender assistant, your task is to process the Assessor's item-by-item evaluation and return only the items that meet the user’s criteria.

    ### Guidelines:

    1. Understand the Inputs:
    - You will receive the user's request and a list of item evaluations from the Assessor.

    2. Filter Based on the Assessor's Evaluation:
    - For each item evaluated as `"Matches the user's criteria"` by the Assessor, include it in your output.
    - Exclude all items evaluated as `"Does not match the user's criteria"`.

    3. Output the Matching Items:
    - Return the exact details of the matching items, including all their attributes (e.g., name, location, stars, review count, category).
    - If no items match, return a concise message:  
        `"No items match the user’s request."`

    4. Be Precise and Minimalistic:
    - Ensure the response contains only the matching items or the short message if no items match.
    - Do not include any additional comments or explanations.
    """,
    shared_memory_read_keys=["extract_info"],
    shared_memory_transition_message=["Below is the user's request:"],
    shared_memory_write_keys=["recommend_items"]
)

exaplainer = Neuron(
    name="exaplainer",
    llm_config=llm_config,
    system_message="""
    You are an Explainer Assistant tasked with providing a clear explanation for the items recommended by the Recommender assistant based on the user's request. Your role is to justify the relevance of recommended items or explain why no items were recommended.

    Inputs:
    1. User Request: A description specifying the user's preferences or criteria.
    2. Recommender Response: A list of recommended items or an indication that no items matched.

    Guidelines:

    1. Recommended Items:
    - Begin with: "For the recommended items, here is an explanation:"
    - For each item, provide:
    - Item Name: [Name]
    - Explanation: [How it meets the user's criteria, emphasizing key attributes like stars, review count, and category.]

    2. No Recommendations:
    - State: "No items were recommended based on the user's request."
    - Follow with: "Here are the unmet criteria:" and list the specific criteria not satisfied.

    3. Output Format:
    - For Recommended Items:
    For the recommended items, here is an explanation:
    Item Name: [Name]
    Explanation: [Details about how it meets the user's request.]

    - For No Recommendations:
    No items were recommended based on the user's request.
    Here are the unmet criteria:
    - [Criterion 1]
    - [Criterion 2]

    Example Outputs:

    Case 1: Recommended Items
    For the recommended items, here is an explanation:
    Item Name: "Wireless Headphones"
    Explanation: "Meets the criteria of 4.5 stars or higher, over 100 reviews, and the 'Electronics' category."

    Item Name: "Bluetooth Earbuds"
    Explanation: "Satisfies the request for portability, high ratings, and the specified 'Electronics' category."

    Case 2: No Recommendations
    No items were recommended based on the user's request.
    Here are the unmet criteria:
    - Minimum rating of 4.0 stars.
    - Category: 'Home Appliances'.

    Tone:
    - Be clear, concise, and focused on the user's criteria.
    - Avoid redundant or overly technical language.
    """,
    shared_memory_read_keys=["extract_info"],
    shared_memory_transition_message=["Below is the user's request:"],
)

evaluator = Neuron(
    name="evaluator",
    llm_config=llm_config,
    system_message="""
   You are an Evaluator Assistant tasked with validating the consistency of the process, which includes the user’s request, recommended items, and the Explainer Assistant explanation. Your goal is to ensure the recommendations and explanations align with the user’s criteria.

    Inputs:
    1. User Request: The user’s criteria (e.g., stars, review count, category). Only consider explicitly mentioned features.
    2. Recommended Items: The items provided by the Recommender assistant.
    3. Explanation: The justification provided by the Explainer assistant.

    Guidelines:

    1. Validate Recommendations:
    - Check if all recommended items meet the user’s explicitly described criteria.
    - Disregard features not mentioned in the user’s request.

    2. Verify Explanation:
    - Confirm that the Explainer assistant explanation accurately reflects the attributes of the recommended items and aligns with the user’s request.
    - Identify any mismatches between the explanation and the recommendations.

    3. Output:
    - If the process is consistent:
    - Respond: "The process is consistent. Here are the validated recommendations and explanations:"
    - List the recommended items with their attributes and the corresponding explanation.
    - If the process is inconsistent:
    - Respond: "The evaluation failed due to the following issues:"
    - Specify:
        - Items that do not meet the user’s criteria.
        - Mismatches between the explanation and the recommendations or user’s request.

    Example Outputs:

    Case 1: Process is Consistent
    "The process is consistent. Here are the validated recommendations and explanations:"
    - Item: "Wireless Headphones"
    Attributes: 4.5 stars, 150 reviews, Category: Electronics
    Explanation: "Meets the criteria of 4.5 stars or higher, over 100 reviews, and belongs to the requested category 'Electronics'."

    - Item: "Bluetooth Earbuds"
    Attributes: 4.8 stars, 200 reviews, Category: Electronics
    Explanation: "Satisfies the request for portability, high ratings, and the specified category 'Electronics'."

    Case 2: Evaluation Failed
    "The evaluation failed due to the following issues:"
    - Item "Bluetooth Earbuds" does not meet the minimum review count.
    - The explanation incorrectly states that all items meet the review count criteria.

    Tone:
    - Be direct and objective.
    - Focus on clarity and relevance to the user’s criteria.
    """,
    shared_memory_read_keys=["extract_info", "recommend_items"],
    shared_memory_transition_message=["Below is the user's request:", "Items recommended:"],
)

In [ ]:
cycle_1 = CycleComponent(name="Cycle_1", neurons=[extract_info_neuron, retriever_neuron, rerank_neuron, assessor_neuron], repetitions=1)
sequential2 = SequentialComponent(name="Seq_1", neurons=[recommender, exaplainer, evaluator])
pipeline = Pipeline()
pipeline.add_edge(cycle_1, sequential2)
pipeline.set_entry_point(cycle_1)

In [ ]:
user.initiate_chat(
    #pipeline, message="I need a Health & Medical clinic with at least 1 stars and more than 5 reviews.",
    pipeline, message="Shopping outlet with at least 3 stars and more than 4 reviews.",
)

In [ ]:
stop here

### Components test

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('../..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

from neuron.runtime_logging import start
logging_session_id = start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

In [ ]:
from neuron.components import SequentialComponent, CycleComponent, Pipeline
from neuron.neurons import User, Neuron, RouterNeuron

llm_config={
    "config_list": [
        {
            "client": "groq",
            "model": "llama3-groq-70b-8192-tool-use-preview",
            "api_key": "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",
            "temperature": 0.0
        }
    ]
}

user = User(name="User", llm_config=llm_config)

common_seq1 = "Dado um numero fornecido pelo usuario, vc deve adicionar mais um a esse numero. Seu output é somente o numero somado, sem texto adicional. Considere a ultima mensagem do user."
common_seq2 = "Dado um numero fornecido pelo usuario, vc deve adicionar mais dois a esse numero. Seu output é somente o numero somado, sem texto adicional. Considere a ultima mensagem do user."

a1 = Neuron(name="Assistante 1", llm_config=llm_config, system_message=common_seq1)
seq1 = SequentialComponent(name="Sequential 1", neurons=[a1])

a0 = Neuron(name="Assistante 0", llm_config=llm_config, system_message="Multiplica o numero fornecido pelo usuario por 2.")
seq0 = SequentialComponent(name="Sequential 0", neurons=[a0])

a2 = Neuron(name="Assistante 2", llm_config=llm_config, system_message=common_seq1)
a3 = Neuron(name="Assistante 3", llm_config=llm_config, system_message=common_seq1)
a4 = Neuron(name="Assistante 4", llm_config=llm_config, system_message=common_seq1)

cycle_router_neuron = Neuron(
    name="Cycle Router",
    llm_config=llm_config,
    system_message = """
    "Evaluate the input based on the number provided by the user. Extract the number if it's part of a larger text. If the number is greater than 10, respond with 'TERMINATE' exactly, with no additional text or explanation. If the number is 10 or less, or if no number is provided, do not respond at all. For example, if the input is 'The number is 15,' reply 'TERMINATE.' If the input is 'The number is 5,' provide no response.
    """,
)
cycle = CycleComponent(name="Cycle 1", neurons=[a2, a3, a4], 
                       repetitions=1,
                       default_component = seq0,
                       cycle_router_neuron= cycle_router_neuron)


a5 = Neuron(name="Assistante 5", llm_config=llm_config, system_message=common_seq1)
seq2 = SequentialComponent(name="Sequential 2", neurons=[a5])

a6 = Neuron(name="Assistante 6", llm_config=llm_config, system_message=common_seq2)
a7 = Neuron(name="Assistante 7", llm_config=llm_config, system_message=common_seq2)
seq3 = SequentialComponent(name="Sequential 3", neurons=[a6, a7])


router1 = RouterNeuron(
    llm_config=llm_config,
    name="router_1",
    route_mapping_function = lambda: {"0": "continue", "1": seq2, "2": seq3},
    system_message = "If the number is even, return '1'. If the number is odd, return '2'. Otherwise, return 'continue'."
)

pipeline = Pipeline()
pipeline.add_edge(seq1, cycle)
pipeline.add_edge(cycle, router1)
pipeline.set_entry_point(seq1)

In [ ]:
user.initiate_chat(
    pipeline, message="Comece de 15"
)